<a href="https://colab.research.google.com/github/danhorsley/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/Copy_of_LS_DS_141_Statistics_Probability_and_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 141
## Statistics, Probability, and Inference

## Prepare - examine what's available in SciPy

As we delve into statistics, we'll be using more libraries - in particular the [stats package from SciPy](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html).

In [0]:
from scipy import stats
dir(stats)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_binned_statistic',
 '_constants',
 '_continuous_distns',
 '_discrete_distns',
 '_distn_infrastructure',
 '_distr_params',
 '_multivariate',
 '_stats',
 '_stats_mstats_common',
 '_tukeylambda_stats',
 'absolute_import',
 'alpha',
 'anderson',
 'anderson_ksamp',
 'anglit',
 'ansari',
 'arcsine',
 'argus',
 'bartlett',
 'bayes_mvs',
 'bernoulli',
 'beta',
 'betaprime',
 'binned_statistic',
 'binned_statistic_2d',
 'binned_statistic_dd',
 'binom',
 'binom_test',
 'boltzmann',
 'boxcox',
 'boxcox_llf',
 'boxcox_normmax',
 'boxcox_normplot',
 'bradford',
 'burr',
 'burr12',
 'cauchy',
 'chi',
 'chi2',
 'chi2_contingency',
 'chisquare',
 'circmean',
 'circstd',
 'circvar',
 'combine_pvalues',
 'contingency',
 'cosine',
 'crystalball',
 'cumfreq',
 'describe',
 'dgamma',
 'dirichlet',
 'distributions',
 'division',
 'dlaplace',
 'dweibull',
 'energy_dist

In [0]:
# As usual, lots of stuff here! There's our friend, the normal distribution
norm = stats.norm()
print(norm.mean())
print(norm.std())
print(norm.var())

0.0
1.0
1.0


In [0]:
# And a new friend - t
t1 = stats.t(5)  # 5 is df "shape" parameter
print(t1.mean())
print(t1.std())
print(t1.var())

0.0
1.2909944487358056
1.6666666666666667


![T distribution PDF with different shape parameters](https://upload.wikimedia.org/wikipedia/commons/4/41/Student_t_pdf.svg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/Student's_t-distribution#/media/File:Student_t_pdf.svg))*

The t-distribution is "normal-ish" - the larger the parameter (which reflects its degrees of freedom - more input data/features will increase it), the closer to true normal.

In [0]:
t2 = stats.t(30)  # Will be closer to normal
print(t2.mean())
print(t2.std())
print(t2.var())

0.0
1.0350983390135313
1.0714285714285714


Why is it different from normal? To better reflect the tendencies of small data and situations with unknown population standard deviation. In other words, the normal distribution is still the nice pure ideal in the limit (thanks to the central limit theorem), but the t-distribution is much more useful in many real-world situations.

History sidenote - this is "Student":

![William Sealy Gosset](https://upload.wikimedia.org/wikipedia/commons/4/42/William_Sealy_Gosset.jpg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/File:William_Sealy_Gosset.jpg))*

His real name is William Sealy Gosset, and he published under the pen name "Student" because he was not an academic. He was a brewer, working at Guinness and using trial and error to determine the best ways to yield barley. He's also proof that, even 100 years ago, you don't need official credentials to do real data science!

## Live Lecture - let's perform and interpret a t-test

We'll generate our own data, so we can know and alter the "ground truth" that the t-test should find. We will learn about p-values and how to interpret "statistical significance" based on the output of a hypothesis test. We will also dig a bit deeper into how the test statistic is calculated based on the sample error, and visually what it looks like to have 1 or 2 "tailed" t-tests.

In [0]:
# TODO - during class, but please help!

## Assignment - apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
from google.colab import files
uploaded = files.upload()

Saving house-votes-84.txt to house-votes-84.txt


In [0]:
import pandas as pd
import numpy as np
df = pd.read_csv('house-votes-84.txt',header = None)
df = df.rename(index=str,columns={0: "Party"})


In [83]:
df.head()

,Party,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,0,1,0,1,1,1,0,0,0,1,?,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,?
2,democrat,?,1,1,?,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,?,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,?,1,1,1,1


In [0]:
# so we will replace the mising votes with the average vote from the members party (roudnign up or down) on that particular issue
#i realise this could skew the data but seems liek best option
for col in range(1,17):
  df[col] = np.where(df[col]=='y',1,df[col])
  df[col] = np.where(df[col]=='n',0,df[col])
# i had to use that replace so as not to change n in republican

df_rep = df[df['Party']=='republican'].copy()
df_dem = df[df['Party']=='democrat'].copy()

df_rep = df_rep.replace('?',np.nan)
df_dem = df_dem.replace('?',np.nan)

In [85]:
df_dem.head()

,Party,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [86]:
rep_vote_means = [round(df_rep[col].mean(),0) for col in range(1,17)]
dem_vote_means = [round(df_dem[col].mean(),0) for col in range(1,17)]

for i in range (1,17):
  df_rep[i] = np.where(df_rep[i].isna(),rep_vote_means[i-1],df_rep[i])
  

for i in range (1,17):
  df_dem[i] = np.where(df_dem[i].isna(),dem_vote_means[i-1],df_dem[i])
  
  
  
df_dem.head()

,Party,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
2,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [95]:

df_clean = df_rep.append(df_dem)
df_clean.shape

(435, 17)

In [0]:
avreps = [(df_rep[col].mean()) for col in range(1,17)] 

In [0]:
avdems = [(df_dem[col].mean()) for col in range(1,17)]


In [125]:
#difference between average voting 
[(avreps[i]-avdems[i]) for i in range(0,len(avreps))]


[-0.43345371856607806,
 0.01116907437132153,
 -0.7604333868378812,
 0.9356607811663992,
 0.7463884430176564,
 0.43813536650615303,
 -0.5468833600856073,
 -0.6886035313001606,
 -0.6621856607811664,
 0.10105671482075973,
 -0.4030898876404494,
 0.7461209202782236,
 0.5956393793472445,
 0.6450642054574638,
 -0.5758426966292134,
 -0.25267522739432857]

In [0]:
#we can see big differences in 3,4 and 5 and 12
import scipy
from scipy import stats

In [139]:
# we can see in vote 3 that democrats voted in favour much more than the republicans.  let's find the tscore for that using our own calc and the scipy calc

mean_vote3 = df_clean[3].mean()
mean_demvote3 = df_dem[3].mean()
std_demvote3 = df_dem[3].std()
nn = len(df_dem[3])

t_stat_vote3 = (mean_demvote3 - mean_vote3)/(std_demvote3/nn**(1/2))
t_stat_vote3

15.39379870588928

In [141]:
#now with scipy - checking tstat and p value in that order
scipy.stats.ttest_1samp(df_dem[3], mean_vote3)[0],'{0:.20f}'.format(scipy.stats.ttest_1samp(df_dem[3], mean_vote3)[1])


(15.393798705889292, '0.00000000000000000000')

In [142]:
#so we can see this satisfies all teh conditions, dems voted for it more, p score very small
#let's try the same for vote 4 where rep average is greater than dem average
mean_vote4 = df_clean[4].mean()
scipy.stats.ttest_1samp(df_rep[4], mean_vote4)[0],'{0:.20f}'.format(scipy.stats.ttest_1samp(df_rep[4], mean_vote4)[1])


(68.4287971497207, '0.00000000000000000000')

In [143]:
#again we can see a very small p value
#now let's look at vote 2 where there didn't seem to be much of a difference in the averages
#here's republicans p scores
mean_vote2 = df_clean[2].mean()
scipy.stats.ttest_1samp(df_rep[2], mean_vote2)[0],'{0:.20f}'.format(scipy.stats.ttest_1samp(df_rep[2], mean_vote2)[1])

(0.17872424527439146, '0.85837086705464415992')

In [144]:
#and now dems
scipy.stats.ttest_1samp(df_dem[2], mean_vote2)[0],'{0:.20f}'.format(scipy.stats.ttest_1samp(df_dem[2], mean_vote2)[1])

(-0.14154187644837354, '0.88754901747324765893')

In [0]:
#here we can see both p scores greater than 0.85 so there isn't mcuh difference between the parties

In [0]:
#i'm not sure entirely of what the stretch goal means, but i'm guessing it means write a function to compare the p scores of two dataframes over a number
#shared factors and we can drop the party name classifier so easier to generalise

def pscore_checker(df1,df2):
  df_appended = df1.append(df2)
  df_appended_means = [(df_appended.iloc[:, i].mean()) for i in range(0,len(df_appended.columns))]
  p_values1=[]
  p_values2=[]
  
  for i in range (0,len(df1.columns)):
    
    p_values1.append('{0:.20f}'.format(scipy.stats.ttest_1samp(df1.iloc[:, i], df_appended_means[i])[1]))
  
  for i in range (0,len(df2.columns)):
    
    p_values2.append('{0:.20f}'.format(scipy.stats.ttest_1samp(df2.iloc[:, i], df_appended_means[i])[1])) 
  
  answer_list = []
  for j in range(0,len(list(df1.columns))):
    bbb = (list(df1.columns)[j],p_values1[j],p_values2[j])
    answer_list.append(bbb)
    
  return answer_list

In [0]:
# dropping party classifier
df1_test = df_rep.copy().drop(['Party'],axis=1)
df2_test = df_dem.copy().drop(['Party'],axis=1)


In [161]:
pscore_checker(df1_test,df2_test)

[(1, '0.00000000000000112536', '0.00000004834971429816'),
 (2, '0.85837086705464415992', '0.88754901747324765893'),
 (3, '0.00000000000000000000', '0.00000000000000000000'),
 (4, '0.00000000000000000000', '0.00000000000000000000'),
 (5, '0.00000000000000000000', '0.00000000000000000000'),
 (6, '0.00000000000000000000', '0.00000007392423923297'),
 (7, '0.00000000000000000017', '0.00000000000000520488'),
 (8, '0.00000000000000000000', '0.00000000000000000000'),
 (9, '0.00000000000000000000', '0.00000000000000000004'),
 (10, '0.10774840374003515076', '0.20295684430899102457'),
 (11, '0.00000000000000000776', '0.00000069103266307509'),
 (12, '0.00000000000000000000', '0.00000000000000000000'),
 (13, '0.00000000000000000000', '0.00000000000000240231'),
 (14, '0.00000000000000000000', '0.00000000000000072040'),
 (15, '0.00000000000000000000', '0.00000000000036388549'),
 (16, '0.00002058651261298255', '0.00000000000030129732')]

In [155]:
# here we can see that column 2 matches our previous calculation

['Party', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

# Resources

- https://homepage.divms.uiowa.edu/~mbognar/applets/t.html
- https://rpsychologist.com/d3/tdist/
- https://gallery.shinyapps.io/tdist/
- https://en.wikipedia.org/wiki/Standard_deviation#Sample_standard_deviation_of_metabolic_rate_of_northern_fulmars